In [ ]:
# Virtual Mouse

import cv2
import numpy as np
import mediapipe as mp
import pyautogui
import math

screen_width ,screen_height = pyautogui.size()

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7,min_tracking_confidence=0.7)

prev_x ,prev_y = 0,0
smoothening = 7

click_threshold = 15
click_down = False


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    
    f = cv2.flip(f,1)
    h,w,_ = f.shape
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            lms = []
            for id,lm in enumerate(hand_landmarks.landmark):
                cx,cy = int(lm.x*w),int(lm.y*h)
                lms.append((id,cx,cy))

            mp_drawing.draw_landmarks(f,hand_landmarks,mp_hands.HAND_CONNECTIONS)
            
            x_index,y_index = lms[8][1],lms[8][2]
            x_thumb,y_thumb = lms[4][1],lms[4][2]

            cv2.circle(f,(x_index,y_index),10,(255,0,0),cv2.FILLED)

            screen_x = np.interp(x_index,[0,w],[0,screen_width])
            screen_y = np.interp(y_index,[0,h],[0,screen_height])

            curr_x = prev_x + (screen_x-prev_x)/smoothening
            curr_y = prev_y + (screen_y-prev_y)/smoothening

            pyautogui.moveTo(curr_x,curr_y)
            prev_x,prev_y = curr_x,curr_y

            dist = math.hypot(x_thumb-x_index,y_thumb-y_index)
            if dist < click_threshold and not click_down:
                click_down  = True
                pyautogui.click()
                cv2.circle(f,(x_index,y_index),15,(0,255,0),cv2.FILLED)
            elif dist>=click_threshold:
                click_down=False



        
    cv2.imshow("Video Frame",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()